# Remark
## インプットファイルの要件
- 元データのCSVが時系列順で[ForwardBG, signal, BackwardBG]の順に並んでいる。
- ch1で奇数番目のファイバー、ch2で偶数番目のファイバーを読んでいる(ch2を使わない場合はch2を無視して良い。)
- ch1のファイバー番号の方がch2のファイバー番号より若い

In [75]:
# csvファイルを読み込んで1次元配列を作る。[1st_fbg,2nd_fbg,1st_sig,2nd_sig,1st_bbg,2nd_bbg]
# 1次元配列から2列の2次元配列を作成。ヘッダは[ch1,ch2]\
# |ch1|ch2|\
# |1st_fbg|2nd_fbg|\
# |1st_sig|2nd_sig|\
# |1st_bbg|2nd_bbg|
# 
# 3n行目の値と3n+2行目の値の平均をとって3n+1行目の値から引く(n=0,1,2,3...)
# その値を新しいリストに詰める。
import glob as glob
import pandas 
import numpy as np 
import csv

outFileName = 'output_NormalOrder.csv'
csv_files = sorted(glob.glob('../logger_data/2023-09-06-155000/*.csv'))

In [84]:
# make 1D array from csv
data_1dim = np.empty(0)
for file in csv_files:
    mydf = pandas.read_csv(file,
                           names=['No.', 'time', 'ms', 'Ch1 [V]', 'Ch2 [V]', 'Ch3 [V]', 'Ch4 [V]', 'Alarm [V]', 'AlarmOut [V]'],
                           usecols=[ 'Ch1 [V]', 'Ch2 [V]' ],
                           encoding="Shift-JIS", 
                           skiprows=21)
    
    mean_ch1: float =0
    mean_ch2: float =0
    mean_ch1 = mydf['Ch1 [V]'].mean(numeric_only=False)
    mean_ch2 = mydf['Ch2 [V]'].mean(numeric_only=False)

    data_1dim = np.append(data_1dim, [mean_ch1,mean_ch2])
nrows = len(data_1dim) // 2
data_2dim = data_1dim.reshape(nrows,2)
print("data_2dim=",data_2dim)


data_2dim= [[0.1894     0.29892   ]
 [1.96016    0.3502    ]
 [0.19776    0.29012   ]
 [0.14217647 0.25041176]
 [2.38515152 0.31254545]
 [0.16608    0.26676   ]
 [0.201      0.319     ]
 [2.15064    0.37004   ]
 [0.20351515 0.30387879]
 [0.19460606 0.29748485]
 [2.02       0.34618182]
 [0.19414286 0.28463265]
 [0.12551515 0.19745455]
 [2.21032308 0.24801538]
 [0.13392    0.19924   ]
 [0.09695385 0.13503077]
 [2.04433333 0.18121212]
 [0.10612195 0.15243902]
 [0.13268    0.20508   ]
 [2.34930769 0.25395385]
 [0.13921212 0.21427273]
 [0.17536    0.27068   ]
 [1.98541463 0.31282927]
 [0.17478788 0.25454545]
 [0.13011765 0.19635294]
 [1.94730303 0.2379697 ]
 [0.13073171 0.18646341]
 [0.08048    0.13168   ]
 [2.44336364 0.18106061]
 [0.09745455 0.15072727]]


In [78]:
# 処理する行数（n）を決定
n = data_2dim.shape[0] // 3
print(n)
outputdata = np.empty(0)

# 3n行目と3n+2行目の平均を計算して3n+1行目から引いて、ndarray型のoutputdataに詰める。
for i in range(n):
    row1 = data_2dim[3 * i]
    row2 = data_2dim[3 * i + 2]
    average = (row1 + row2) / 2
    outputdata = np.append(outputdata,data_2dim[3 * i + 1]-average)
    #print(data_2dim[3*i+1])
print(outputdata)


10
[1.76658    0.05568    2.23102328 0.05395957 1.94838242 0.05860061
 1.82562554 0.05512307 2.0806055  0.04966811 1.94279543 0.03747722
 2.21336163 0.04427748 1.81034069 0.05021654 1.81687835 0.04656152
 2.35439636 0.03985697]


In [79]:
#データフレームからCSVに書き出す。
df = pandas.DataFrame(outputdata,columns=['fiber signals [V]'])
df.to_csv(outFileName, index=False)